In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25985
2,Huelva,Confirmados PDIA 14 días,894
3,Huelva,Tasa PDIA 14 días,"174,21127501607654"
4,Huelva,Confirmados PDIA 7 días,491
5,Huelva,Total Confirmados,26186
6,Huelva,Curados,21382
7,Huelva,Fallecidos,355


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25985.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6794.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1383 personas en los últimos 7 días 

Un positivo PCR cada 922 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25985.0,491.0,894.0,513170.0,95.679794,174.211275,178.0
Condado-Campiña,7662.0,278.0,535.0,156231.0,177.941638,342.441641,100.0
Huelva-Costa,15554.0,183.0,308.0,289548.0,63.201956,106.372691,70.0
Huelva (capital),6794.0,104.0,156.0,143837.0,72.304066,108.456100,46.0
Almonte,1084.0,79.0,159.0,24507.0,322.356878,648.794222,39.0
Moguer,919.0,40.0,59.0,21867.0,182.924041,269.812960,20.0
Cartaya,1037.0,39.0,58.0,20083.0,194.194095,288.801474,11.0
Palos de la Frontera,500.0,23.0,38.0,11742.0,195.878045,323.624595,8.0
Bollullos Par del Condado,693.0,29.0,58.0,14387.0,201.570863,403.141725,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Bartolomé de la Torre,249.0,19.0,36.0,3761.0,505.184791,957.192236,5.0
Rociana del Condado,389.0,20.0,65.0,7939.0,251.920897,818.742915,4.0
Almonte,1084.0,79.0,159.0,24507.0,322.356878,648.794222,39.0
Alosno,152.0,2.0,19.0,3933.0,50.851767,483.091787,1.0
Bollullos Par del Condado,693.0,29.0,58.0,14387.0,201.570863,403.141725,7.0
Niebla,177.0,4.0,16.0,4116.0,97.181730,388.726919,2.0
Condado-Campiña,7662.0,278.0,535.0,156231.0,177.941638,342.441641,100.0
Palos de la Frontera,500.0,23.0,38.0,11742.0,195.878045,323.624595,8.0
Bonares,199.0,16.0,19.0,6060.0,264.026403,313.531353,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,152.0,2.0,19.0,3933.0,50.851767,483.091787,1.0,0.105263
Isla Cristina,2013.0,2.0,8.0,21393.0,9.348852,37.395410,0.0,0.250000
Niebla,177.0,4.0,16.0,4116.0,97.181730,388.726919,2.0,0.250000
Ayamonte,1248.0,3.0,12.0,21104.0,14.215315,56.861259,1.0,0.250000
Gibraleón,552.0,6.0,20.0,12737.0,47.106854,157.022847,2.0,0.300000
Rociana del Condado,389.0,20.0,65.0,7939.0,251.920897,818.742915,4.0,0.307692
Punta Umbría,672.0,6.0,14.0,15355.0,39.075220,91.175513,0.0,0.428571
Valverde del Camino,432.0,4.0,9.0,12750.0,31.372549,70.588235,1.0,0.444444
Paterna del Campo,123.0,4.0,9.0,3457.0,115.707261,260.341336,3.0,0.444444
